For detailed explanation of all the steps, visit this link: https://huggingface.co/learn/nlp-course/chapter3/1?fw=tf

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Preprocessing

In [6]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/sms-spam.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8c7554842cefc93e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Remove the "Unnamed: 0" column from the train split
dataset["train"] = dataset["train"].remove_columns(["Unnamed: 0"])

dataset

DatasetDict({
    train: Dataset({
        features: ['spam', 'text'],
        num_rows: 4837
    })
})

In [8]:
raw_train_dataset = dataset["train"]
raw_train_dataset[0]

{'spam': 0,
 'text': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'}

In [9]:
raw_train_dataset.features

{'spam': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None)}

In [10]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# tokenized_dataset = tokenizer(
#     dataset["train"]["text"],
#     padding=True,
#     truncation=True,
# )

In [11]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [12]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/4837 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['spam', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4837
    })
})

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [14]:
samples = tokenized_datasets["train"][:8]
[len(x) for x in samples["input_ids"]]

[34, 17, 56, 20, 19, 52, 20, 50]

In [15]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'spam': TensorShape([8]),
 'text': TensorShape([8]),
 'input_ids': TensorShape([8, 56]),
 'token_type_ids': TensorShape([8, 56]),
 'attention_mask': TensorShape([8, 56])}

In [16]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["spam"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


# Training

In [19]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [20]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model.fit(tf_train_dataset, epochs=3)

Epoch 1/3
605/605 [==============================] - 158s 181ms/step - loss: 0.0664 - accuracy: 0.9793
Epoch 2/3
605/605 [==============================] - 84s 139ms/step - loss: 0.0170 - accuracy: 0.9959
Epoch 3/3
605/605 [==============================] - 89s 146ms/step - loss: 0.0060 - accuracy: 0.9992
